# Grounding in Logic Tensor Networks (LTN)

## Real Logic

The semantics of LTN differs from the standard abstract semantics of First-order Logic (FOL) in that domains are interpreted concretely by tensors in the Real field.
To emphasize the fact that LTN interprets symbols which are grounded on real-valued features, we use the term *grounding*, denoted by $\mathcal{G}$, instead of interpretation. 
$\mathcal{G}$ associates a tensor of real numbers to any term of the language, and a real number in the interval $[0,1]$ to any formula $\phi$. 
In the rest of the tutorials, we commonly use "tensor" to designate "tensor in the Real field".

The language consists of a non-logical part (the signature) and logical connectives and quantifiers.
* **constants** denote individuals from some space of tensors $\bigcup\limits_{n_1 \dots n_d \in \mathbb{N}^*} \mathbb{R}^{n_1 \times \dots \times n_d}$ (tensor of any rank). The individual can be pre-defined (data point) or learnable (embedding).
* **variables** denote sequence of individuals.
* **functions** can be any mathematical function either pre-defined or learnable. Examples of functions can be distance functions, regressors, etc. Functions can be defined using any operations in Tensorflow. They can be linear functions, Deep Neural Networks, and so forth.
* **predicates** are represented as mathematical functions that map from some n-ary domain of individuals to a real from $[0,1]$ that can be interpreted as a truth degree. Examples of predicates can be similarity measures, classifiers, etc.
* **connectives** -- not, and, or, implies -- are modeled using fuzzy semantics.
* **quantifiers** are defined using aggregators.

This tutorial explains how to ground constants, variables, functions and predicates.

In [1]:
import logictensornetworks as ltn
import torch
import torch.nn as nn
from torch.optim import Adam
import numpy as np

## Constants

LTN constants are grounded as some real tensor. Each constant $c$ is mapped to a point in $\mathcal{G}(c) \in \bigcup\limits_{n_1 \dots n_d \in \mathbb{N}^*} \mathbb{R}^{n_1 \times \dots \times n_d}$. Notice that the objects in the domain may be tensors of any rank. A tensor of rank $0$ corresponds to a scalar, a tensor of rank $1$ to a vector, a tensor of rank $2$ to a matrix and so forth, in the usual way.  
Here we define $\mathcal{G}(c_1)=(2.1,3)$ and $\mathcal{G}(c_2)=\begin{pmatrix}
4.2 & 3 & 2.5\\
4 & -1.3 & 1.8
\end{pmatrix}$.

In [15]:
c1 = ltn.constant([2.1,3])
x = ltn.constant(2)
c2 = ltn.constant([[4.2,3,2.5],[4,-1.3,1.8]])

Note that a constant can be set as learnable by using the optional argument `trainable=True`. This is useful to learn embeddings for some individuals.
The features of the tensor will be considered as trainable parameters (learning in LTN is explained in a further notebook).

In [3]:
c3 = ltn.constant([0.,0.], trainable=True)

LTN constants are implemented with Tensorflow constants when non trainable, and with Tensorflow variables when trainable, with some added dynamic attributes. That means we can query the value of the constant easily.

In [4]:
print(c1)
print(c1.detach().numpy())
print(c3)
print(c3.detach().numpy())

tensor([2.1000, 3.0000])
[2.1 3. ]
tensor([0., 0.], requires_grad=True)
[0. 0.]


## Predicates

LTN Predicates are grounded in mappings that assign a value between zero and one to some n-ary space of input values. Predicates in LTN can be neural networks or any other function that achieves such a mapping.  

There are different ways to construct a predicate in LTN:
- the default constructor `ltn.Predicate(model)` takes in argument a `tf.keras.Model` instance; it can be used to ground any custom function (succession of operations, Deep Neural Network, ...) that return a value in $[0,1]$,
- the lambda constructor `ltn.Predicate.Lambda(function)` takes in argument a lambda function; it is appropriate for small mathematical operations with **no trainable weights** (non-trainable function) that return a value in $[0,1]$.

The following defines a predicate $P_1$ using the similarity to the point $\vec{\mu}=\left<2,3\right>$ with $\mathcal{G}(P_1):\vec{x}\mapsto \exp(-\|\vec{x}-\vec{\mu} \|^2)$, and a predicate $P_2$ defined using a Tensorflow model.

In [5]:
mu = torch.tensor([2.,3.])
P1 = ltn.Predicate.Lambda(lambda x: torch.exp(-torch.norm(x-mu,dim=1)))

class ModelP2(nn.Module):
    def __init__(self, input_size):
        super(ModelP2, self).__init__()
        self.fc1 = nn.Linear(input_size,5)
        self.fc2 = nn.Linear(5,1)
        self.elu = nn.ELU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.elu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

modelP2 = ModelP2(2)
P2 = ltn.Predicate(modelP2)

One can easily query predicates using LTN constants and LTN variables (see further in this notebook to query using variables).

In [6]:
c1 = ltn.constant([2.1,3])
c2 = ltn.constant([4.5,0.8])
print(P1(c1))
print(P1(c2))

print(P2(c1)) # The first call of a tf model initializes its input layers.
              # After this line, P2 is a model with inputs in R^2.

tensor([0.9048])
tensor([0.0358])
tensor([[0.4588]], grad_fn=<SigmoidBackward>)


NOTE:  
- If an LTN predicate (or an LTN function) takes several inputs, e.g. $P_3(x_1,x_2)$, the arguments must be passed as a list (cf Tensorflow's conventions).
- LTN converts inputs such that there is a "batch" dimension on the first axis. Therefore, most operations should work with `axis=1`.

In [7]:
class ModelP3(nn.Module):
    def __init__(self, input_sizes):
        super(ModelP3, self).__init__()
        input_size = sum(input_sizes)
        self.fc1 = nn.Linear(input_size,5)
        self.fc2 = nn.Linear(5,1)
        self.elu = nn.ELU()
        self.sigmoid = nn.Sigmoid()

    def forward(self,inputs):
        x1, x2 = inputs[0], inputs[1] # multiple arguments are passed as a list
        x = torch.cat([x1,x2], dim=1) # dim=0 is the batch dimension
        x = self.fc1(x)
        x = self.elu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

    
P3 = ltn.Predicate(ModelP3(input_sizes=[2,2]))
c1 = ltn.constant([2.1,3])
c2 = ltn.constant([4.5,0.8])

print(P3([c1,c2])) # multiple arguments are passed as a list

tensor([[0.4457]], grad_fn=<SigmoidBackward>)


One case define trainable or non trainable 0-ary predicates (propositional variables) using `ltn.proposition`. They are grounded as a mathematical constant in $[0,1]$.

In [8]:
# Declaring a trainable 0-ary predicate with initial truth value 0.3
A = ltn.proposition(0.3, trainable=True)

For more details and useful ways to create predicates (incl. how to integrate multiclass classifiers as binary predicates), see the complementary notebook.

## Functions

LTN functions are grounded as any mathematical function that maps $n$ individuals to one individual in the tensor domains.  

There are different ways to construct an LTN function in LTN:
- the default constructor `ltn.Function(model)` takes in argument a `tf.keras.Model` instance; it can be used to ground any custom function (succession of operations, Deep Neural Network, ...),
- the lambda constructor `ltn.Function.Lambda(function)` takes in argument a lambda function; it is appropriate for small mathematical operations with **no weight tracking** (non-trainable function).

The following defines the grounding of a function $f_1$ that computes the difference of two inputs with $\mathcal{G}(f_1):\vec{u},\vec{v}\mapsto \vec{u} - \vec{v}$ and a function $f_2$ that uses a deep neural network that projects a value to $\mathbb{R}^5$.

In [9]:
f1 = ltn.Function.Lambda(lambda args: args[0]-args[1])

class MyModel(nn.Module):
    def __init__(self, input_size):
        super(MyModel, self).__init__()
        self.fc1 = nn.Linear(input_size,4)
        self.fc2 = nn.Linear(4,5)
        self.relu = nn.ReLU()

    def forward(self,x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

    
model_f2 = MyModel(2)
f2 = ltn.Function(model_f2)

One can easily query predicates using LTN constants and LTN variables (see further in this notebook to query using variables).

In [10]:
c1 = ltn.constant([2.1,3])
c2 = ltn.constant([4.5,0.8])
print(f1([c1,c2])) # multiple arguments are passed as a list
print(f2(c1))

tensor([[-2.4000,  2.2000]])
tensor([[-0.8495, -0.1782, -0.9198,  0.5832, -0.1980]],
       grad_fn=<AddmmBackward>)


## Variables

LTN variables are sequences of individuals/constants from a domain. Variables are useful to write quantified statements, e.g. $\forall x\ P(x)$. Notice that a variable is a sequence and not a set; the same value can occur twice in the sequence.

The following defines two variables $x$ and $y$ with respectively 10 and 5 individuals, sampled from normal distributions in $\mathbb{R}^2$.  
In LTN, variables need to be labelled (see the arguments `'x'` and `'y'` below).

In [11]:
x = ltn.variable('x',np.random.normal(0.,1.,(10,2)))
y = ltn.variable('y',np.random.normal(0.,4.,(5,2)))

Evaluating a term/predicate with one variable of $n$ individuals yields $n$ output values, where the $i$-th output value corresponds to the term calculated with the $i$-th individual.

Similarly, evaluating a term/predicate with $k$ variables $(x_1,\dots,x_k)$, with respectively $n_1,\dots,n_k$ individuals each, yields a result with $n_1 \times \dots \times n_k$ values. The result is organized in a tensor where the first $k$ dimensions can be indexed to retrieve the outcome(s) that correspond to each variable. The tensor has a dynamically added attribute `active_doms` that tells which axis corresponds to which variable (using the label of the variable).

In [12]:
# Notice that the outcome is a 2 dimensional tensor where each cell
# represents the satisfiability of P3 with each individual in x and in y.
res1 = P3([x,y])
print(res1.shape)
print(res1.active_doms) # dynamically added attribute; tells that axis 0 corresponds to x and axis 1 to y 
print(res1[2,0]) # gives the result calculated with the 3rd individual in x and the 1st individual in y

torch.Size([10, 5])
['x', 'y']
tensor(0.5137, grad_fn=<SelectBackward>)


In [14]:
# Notice that the last axe(s) correspond to the dimensions of each outcome;
# here, f2 projects to outcomes in R^2, so the outcome has one additional axis of dimension 2.
res2 = f1([x,y])
print(res2.shape)
print(res2.active_doms)
print(res2[2,0]) # gives the result calculated with the 3rd individual in x and the 1st individual in y

TypeError: expected Tensor as element 1 in argument 0, but got list

In [1]:
c1 = ltn.constant([2.1,3])
res3 = P3([c1,y])
print(res3.shape) # Notice that no axis is associated to a constant.
print(res3.active_doms)
print(res3[0]) # gives the result calculated with c1 and the 1st individual in y

NameError: name 'ltn' is not defined

### Variables made of trainable constants

A variable can be instantiated using two different types of objects:
- A value (numpy, python list, ...) that will be fed in a `tf.constant` (the variable refers to a new object).
- A `tf.Tensor` instance that will be used directly as the variable (the variable refers to the same object).

The latter is useful when the variable denotes a sequence of trainable constants.

In [14]:
c1 = ltn.constant([2.1,3], trainable=True)
c2 = ltn.constant([4.5,0.8], trainable=True)

with tf.GradientTape() as tape:
    # Notice that the assignation must be done within a tf.GradientTape.
    # Tensorflow will keep track of the gradients between c1/c2 and x.
    # Read tutorial 3 for more details.
    x = ltn.variable("x",tf.stack([c1,c2]))
    res = P2(x)
tape.gradient(res,c1).numpy() # the tape keeps track of gradients between P2(x), x and c1

array([-0.02226352, -0.00509561], dtype=float32)